In [17]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from psycopg2 import sql
import pandas as pd
from pyvis.network import Network
from Levenshtein import distance
from itertools import *
import socket
import jpysocket
from threading import Thread
from time import sleep

pd.options.display.float_format = '{:12.3e}'.format

# Recommendation system parameters

In [2]:
max_related_tags = 5 #maximum number of edges of node in graph
max_levenshtein_distance = 4 #maximum distance between two string tags to call them similar
binary_search_check_radius = 5 #levenshtein distance radius check for best
graph_recommendation_depth = 2 #recommendation depth
graph_recommendation_weights = (10,3,1) #recommendation weights (graph_recommendation_depth + 1)
graph_recommendation_weight_title = 10 #recommendation weight for title corresponding

## Connect to postgresql database

In [3]:
conn = psycopg2.connect(dbname='yedom', user='postgres', 
                        password='root', host='localhost')
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = conn.cursor()

## Create 'coursers', 'tags' tables if not exist or clear if exist

In [4]:
try:
    cursor.execute(
        '''CREATE TABLE coursers (
                    id SERIAL PRIMARY KEY,
                    title VARCHAR (255) NOT NULL,
                    author VARCHAR (255) NOT NULL,
                    views INTEGER NOT NULL,
                    likes INTEGER NOT NULL,
                    sponsors TEXT NOT NULL,
                    tags TEXT NOT NULL
        )''')
except:
    cursor.execute('TRUNCATE TABLE coursers RESTART IDENTITY CASCADE')
    
try:
    cursor.execute(
        '''CREATE TABLE tags (
                    id SERIAL PRIMARY KEY,
                    tag VARCHAR(255) UNIQUE NOT NULL,
                    coursers_count INTEGER NOT NULL,
                    related_tags TEXT
        )''')
except:
    cursor.execute('TRUNCATE TABLE tags RESTART IDENTITY CASCADE')

In [5]:
#'coursers' table

def get_tags():
    cursor.execute("""SELECT tags FROM coursers""")
    return [t for t, in cursor]

def get_val_by_id_coursers(ID, val):
    cursor.execute(
            sql.SQL("""SELECT {} FROM coursers WHERE id=%s""")
                .format(sql.Identifier(val)), (int(ID),)
        )
    return cursor.fetchone()[0]

In [6]:
#'tags' table

def get_val_by_tag(tag, val):
    cursor.execute(
            sql.SQL("""SELECT {} FROM tags WHERE tag=%s""")
                .format(sql.Identifier(val)), (tag,)
        )
    return cursor.fetchone()[0]
def get_val_by_id(ID, val):
    cursor.execute(
            sql.SQL("""SELECT {} FROM tags WHERE id=%s""")
                .format(sql.Identifier(val)), (int(ID),)
        )
    return cursor.fetchone()[0]
def best_related_tags(unique_ids):
    coursers_count = [get_val_by_id(ID, 'coursers_count') for ID in unique_ids]
    pairs = list(zip(coursers_count, list(unique_ids)))
    pairs.sort(reverse=True)
    return set([pairs[i][1] for i in range(min(max_related_tags, len(pairs)))])
def related_tags_push(tags):
    ID_tags = [get_val_by_tag(tag, 'id') for tag in tags]
    for ID in ID_tags:
        unique_ids = set([])
        related_tags = get_val_by_id(ID, 'related_tags')
        if related_tags != '':
            unique_ids = set(map(int, related_tags.split(',')))
        unique_ids.update([int(id_tag) for id_tag in ID_tags if id_tag != ID])
        unique_ids = best_related_tags(unique_ids)
        
        cursor.execute("""UPDATE tags SET related_tags=%s WHERE id=%s""", 
                       (','.join(map(str, unique_ids)), ID,))
def add_tag(tag):
    try:
        cursor.execute(
            """INSERT INTO tags (tag, coursers_count, related_tags) VALUES (%s, %s, %s)""", (tag, 1, '',))
    except:
        cursor.execute("""UPDATE tags SET coursers_count=%s WHERE tag=%s""", 
                       (get_val_by_tag(tag, 'coursers_count')+1, tag,))
        cursor.execute("""SELECT setval('tags_id_seq', MAX(id), true) FROM tags""")

## Insert rows from 'examples/*.csv' file to 'coursers' database

In [7]:
df = pd.read_csv('examples/coursers.csv')
df.head()

,title,author,views,likes,sponsors,tags
0,Введение в теорию вероятностей,MrAlexeiMK,10,100,nan,теория вероятностей@математика@вероятности
1,Теория диффиренциальных исчислений,Tester,10,100,nan,дифферинциальные исчисления@математика@диффери...
2,Программирование на C#,Шарпер,10,100,nan,программирование@C#@Visual Studio 2022@Nugget@...
3,Программирование на Java,Джавист,10,100,nan,программирование@Java@Java 17@Intellij Idea 20...
4,Программирование на Python,Питонщик,10,100,nan,программирование@Python@Python 3@Jupyter Noteb...


In [8]:
for index, row in df.iterrows():
    cursor.execute(
        """INSERT INTO coursers (title, author, views, likes, sponsors, tags) 
            VALUES (%s, %s, %s, %s, %s, %s)""", 
        (row['title'], row['author'], row['views'], row['likes'], row['sponsors'], row['tags']))

## Update 'tags' database (everything below needs to be update in period)

In [9]:
for row in get_tags():
    tags = row.split('@')
    for tag in tags:
        add_tag(tag)
    related_tags_push(tags)

# Graph creation and visualization

In [10]:
df_tags = pd.read_sql_query("SELECT * from tags ORDER BY id", con=conn)
df_tags.head()

,id,tag,coursers_count,related_tags
0,1,теория вероятностей,1,"2,3"
1,2,математика,2,"1,3,4,5"
2,3,вероятности,1,"1,2"
3,4,дифферинциальные исчисления,1,"2,5"
4,5,дифферинциальные уравнения,1,"2,4"


In [11]:
df_coursers = pd.read_sql_query("SELECT * from coursers ORDER BY id", con=conn)
df_coursers.head()

,id,title,author,views,likes,sponsors,tags
0,1,Введение в теорию вероятностей,MrAlexeiMK,10,100,NaN,теория вероятностей@математика@вероятности
1,2,Теория диффиренциальных исчислений,Tester,10,100,NaN,дифферинциальные исчисления@математика@диффери...
2,3,Программирование на C#,Шарпер,10,100,NaN,программирование@C#@Visual Studio 2022@Nugget@...
3,4,Программирование на Java,Джавист,10,100,NaN,программирование@Java@Java 17@Intellij Idea 20...
4,5,Программирование на Python,Питонщик,10,100,NaN,программирование@Python@Python 3@Jupyter Noteb...


In [12]:
id_by_tag = list(zip(df_tags.tag,df_tags.id))
id_by_tag.sort(key=lambda t: t[0].lower())

In [13]:
G = Network(notebook=True, directed=True)
G.repulsion()

maxID = 1 #restrictive line between 'tags' and 'coursers'

#add nodes
for index, row in df_tags.iterrows():
    maxID = max(maxID, row['id'])
    G.add_node(row['id'], label=row['tag'])
for index, row in df_coursers.iterrows():
    G.add_node(maxID + row['id'], label=row['title'] + " (" + row['author'] + ")", color='green')

#add edges
for index, row in df_tags.iterrows():
    tags = row['related_tags'].split(',')
    for tag in tags:
        G.add_edge(row['id'], int(tag))
for index, row in df_coursers.iterrows():
    tags = row['tags'].split('@')
    for tag in tags:
        fr = int(get_val_by_tag(tag, 'id'))
        to = maxID + row['id']
        G.add_edge(fr, to)

G.show('graph.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


**Используемая для хранения графа память**

Используемая для хранения графа память равна $O(V+E) = O((tagsCount + coursersCount) + (tagsCount + coursersCount)*(maxRelatedTags + coursersCount))$

In [14]:
cc = [1000, 2000, 5000, 1e4, 5e4, 2e5, 5e5, 1e6]
tc = [(20 + 10*i)*cc[i] for i in range(len(cc))]
av = [int(tc[i]//cc[i]) for i in range(len(cc))]

res = [tc[i] + cc[i] + (tc[i] + cc[i]) * (max_related_tags + av[i]) for i in range(len(cc))]
frame = pd.DataFrame({'coursersCount': cc, 'tagsCount': tc, 'averageCoursersOnTag': av, 'Memory': res})
print("maxRelatedTags =", max_related_tags)
frame.to_html(open('memoryInfo.html', 'w'))
frame.head(10)

maxRelatedTags = 5


,coursersCount,tagsCount,averageCoursersOnTag,Memory
0,1.000e+03,2.000e+04,20,5.460e+05
1,2.000e+03,6.000e+04,30,2.232e+06
2,5.000e+03,2.000e+05,40,9.430e+06
3,1.000e+04,5.000e+05,50,2.856e+07
4,5.000e+04,3.000e+06,60,2.013e+08
5,2.000e+05,1.400e+07,70,1.079e+09
6,5.000e+05,4.000e+07,80,3.483e+09
7,1.000e+06,9.000e+07,90,8.736e+09


# Recommendation system on user input from graph

In [15]:
def input_processing(s):
    s = ' '.join(s.split())
    return s

def distance_tags(tag1, tag2):
    return distance(tag1, tag2, weights=(1,1,2))

#O(N + N*(N-1) + N*(N-1)*(N-2))
def splitter(s, maxLen = 3, debug=False):
    arr = s.split(' ')
    N = len(arr)
    tags = []
    for k in range(1, maxLen+1):
        for p in permutations(arr, k):
            tag = ' '.join(p)
            bs = binary_search(tag, debug)
            if bs[0] <= max_levenshtein_distance:
                tags.append(bs)
    tags.sort()
    if debug:
        print(tags)
    return set([t[1] for t in tags])

#O(log(len(id_by_tag)))
def binary_search(tag, debug=False):
    l = 0
    r = len(id_by_tag)
    if debug:
        print(tag+":")
    while r - l > 1:
        mid = (l+r)//2
        if debug:
            print(str("    "), id_by_tag[mid][0])
        if id_by_tag[mid][0].lower() > tag.lower():
            r = mid
        else:
            l = mid
    M = l
    minDis = distance_tags(tag, id_by_tag[l][0])
    
    for m in range(max(0, l-binary_search_check_radius), min(len(id_by_tag), l+binary_search_check_radius+1)):
        dis = distance_tags(tag, id_by_tag[m][0])
        if dis < minDis:
            minDis = dis
            M = m
    if debug:
        print(str("  * "), id_by_tag[M][0], minDis)
    return (minDis, id_by_tag[M][1])

## Getting closest tags

In [16]:
s = input()
s = input_processing(s)

IDS = [list(splitter(s, debug=False))]
print("Closest tags (ids):", IDS)

python
Closest tags (ids): [[19]]


## Add additional tags into 'IDS' from graph

In [17]:
uniques = set(IDS[0])

for i in range(graph_recommendation_depth):
    depth_ids = []
    for node in set(IDS[i]):
        for to in G.neighbors(node):
            if to <= maxID:
                if to not in uniques:
                    depth_ids.append(to)
                    uniques.add(to)
    IDS.append(depth_ids)
print(IDS)
print([[get_val_by_id(ID, 'tag') for ID in d_ids ] for d_ids in IDS])

[[19], [6, 20, 21, 23, 24], [10, 58, 43, 44, 45, 78]]
[['Python'], ['программирование', 'Python 3', 'Jupyter Notebook', 'numpy', 'pandas'], ['для новичков', 'алгоритмы', 'open-source', 'Contributing', 'fork', 'графики']]


## Recommend coursers by graph

In [16]:
cDict = dict()
spl = list(map(str.lower, s.split(' ')))

#tags weight
for i in range(len(IDS)):
    d_ids = IDS[i]
    weight = graph_recommendation_weights[i] #course recommendation weight
    for node in d_ids:
        for to in G.neighbors(node):
            if to > maxID:
                ID = to - maxID #course ID
                
                try:
                    cDict[ID] += weight
                except:
                    cDict[ID] = weight

#title weight
for ID in cDict:
    title = get_val_by_id_coursers(ID, 'title').lower()
    for word in spl:
        if word in title:
            cDict[ID] += graph_recommendation_weight_title

#result
weights_pairs = [(cDict[key], key) for key in cDict]
weights_pairs.sort(reverse=True)

response = [t[1] for t in weights_pairs]
print(weights_pairs)
print(response)

print()
print('Рекомендуем курсы:')
print("    ", [get_val_by_id_coursers(ID, 'title') for ID in response])

NameError: name 's' is not defined

# Python WebSocket server

In [16]:
def recommend_courses(inputStr):
    inputStr = input_processing(inputStr)
    
    #extract tags from processed inputStr
    IDS = [list(splitter(inputStr, debug=False))]
    
    #add additional tags from graph
    uniques = set(IDS[0])

    for i in range(graph_recommendation_depth):
        depth_ids = []
        for node in set(IDS[i]):
            for to in G.neighbors(node):
                if to <= maxID:
                    if to not in uniques:
                        depth_ids.append(to)
                        uniques.add(to)
        IDS.append(depth_ids)
    
    cDict = dict()
    spl = list(map(str.lower, inputStr.split(' ')))
    
    #tags weight
    for i in range(len(IDS)):
        d_ids = IDS[i]
        weight = graph_recommendation_weights[i] #course recommendation weight
        for node in d_ids:
            for to in G.neighbors(node):
                if to > maxID:
                    ID = to - maxID #course ID
                    
                    try:
                        cDict[ID] += weight
                    except:
                        cDict[ID] = weight

    #title weight
    for ID in cDict:
        title = get_val_by_id_coursers(ID, 'title').lower()
        for word in spl:
            if word in title:
                cDict[ID] += graph_recommendation_weight_title

    #result
    weights_pairs = [(cDict[key], key) for key in cDict]
    weights_pairs.sort(reverse=True)

    response = [t[1] for t in weights_pairs]
    
    return response

In [ ]:
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = "localhost"
port = 2003
soc.bind((host, port))
soc.listen(10)

try:
    while True:
        conn, addr = soc.accept()
        
        print("User connected:", addr)
        msg = jpysocket.jpydecode(conn.recv(1024))
        print("Message:", msg)
        IDS = recommend_courses(msg)
        print(IDS)
        response = jpysocket.jpyencode(','.join(map(str, IDS)))
        print(response)
        conn.send(response)
        
        conn.close()
except KeyboardInterrupt:
    pass

User connected: ('127.0.0.1', 19236)
Message: python
[5, 14, 6, 9, 13, 8, 3, 4, 12, 11]
b'\x00\x175,14,6,9,13,8,3,4,12,11'
User connected: ('127.0.0.1', 19242)
Message: нейронные сети
[6, 5, 14, 9, 13, 8, 3, 4, 12, 11]
b'\x00\x176,5,14,9,13,8,3,4,12,11'
User connected: ('127.0.0.1', 19246)
Message: алгоритмы
[13, 12, 5, 4, 14, 6, 8, 3, 9, 11]
b'\x00\x1713,12,5,4,14,6,8,3,9,11'
User connected: ('127.0.0.1', 1048)
Message: саша лох
[]
b'\x00\x00'
